<a href="https://colab.research.google.com/github/01010010obin/miniGPT/blob/main/FinalBigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [3]:
device

'cuda'

In [4]:
torch.manual_seed(1337)

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-04-08 15:45:27--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-04-08 15:45:27 (22.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [8]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [9]:
data = torch.tensor(encode(text), dtype = torch.long)

In [10]:
data[:20], data.shape

(tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
         53, 56]),
 torch.Size([1115394]))

In [11]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
def get_batch(split):

  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  x, y = x.to(device), y.to(device)

  return x, y

In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


Positional embedding, özellikle transformer modellerinde kullanılan bir tekniktir. Temel amacı, bir dizinin (örneğin bir metindeki kelimelerin veya karakterlerin) sırasını modele açıkça belirtmektir. Çünkü transformerlar kendiliğinden dizilim sırasını anlamaz; bunun yerine, giriş verisinin sırasını anlamlandırabilmek için pozisyon bilgisine ihtiyaç duyar.

Dil modellerinde giriş olarak bir token dizisi (kelime, karakter vb.) verildiğinde, transformerlar bu tokenleri bireysel birimler olarak işler. Ancak:

Tokenlerin dizilim sırası, bağlam ve anlam açısından oldukça önemlidir.

Örneğin: "Kedi ağaca tırmandı" ve "Ağaca kedi tırmandı" aynı kelimelerden oluşsa da, sıralama anlamı tamamen değiştirir.

Positional embedding, her tokenin bulunduğu konumun bilgisini modele kodlayarak, bu sıralama bilgisini modele iletir.

Statik Positional Encoding (Sinüs ve Kosinüs Tabanlı):

Learnable Positional Embedding:

In [14]:
train_data.shape

torch.Size([1003854])

In [15]:
xb, yb = get_batch('train')
xb.shape

torch.Size([64, 256])

In [16]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias = False)
    self.query = nn.Linear(n_embd, head_size, bias = False)
    self.value = nn.Linear(n_embd, head_size, bias =False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)

    wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim = -1)
    wei = self.dropout(wei)

    v = self.value(x)

    out = wei @ v

    return out



In [17]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)
    out = self.dropout(self.proj(out))

    return out

In [18]:
class FeedFoward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
        )
  def forward(self, x):
    return self.net(x)


In [19]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedFoward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [20]:
class FinalModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [22]:
model = FinalModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


In [23]:

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [24]:
for iter in range(max_iters):


    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2209, val loss 4.2191
step 500: train loss 1.7466, val loss 1.8977
step 1000: train loss 1.3994, val loss 1.6079
step 1500: train loss 1.2701, val loss 1.5258
step 2000: train loss 1.1903, val loss 1.5021
step 2500: train loss 1.1270, val loss 1.4950
step 3000: train loss 1.0738, val loss 1.4755
step 3500: train loss 1.0198, val loss 1.5056
step 4000: train loss 0.9669, val loss 1.5148
step 4500: train loss 0.9167, val loss 1.5451
step 4999: train loss 0.8666, val loss 1.5626


In [25]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


But with prison: I will stead for my husband.
Alas Canster, altogether with a senator,
Take as my good first ashes: and myself my stand,
Who is enough the labour, the name of horse;
Hold, as misery he cannot do throw me
Than can my sort and vengeance myself.
See I know my craduce coupw hither!

Third Citizen:
Go, do not him go.

MENENIUS:
Those warms that shall warrant you follow.

COMINIUS:
Has it not disposite to them?

BRUTUS:
'Twill instake the likes.

MENENIUS:
We the first: the main goes a
